FINAL PROJECT

In [85]:
# Import Libraries
import pandas as pd
import ast
import numpy as np

In [86]:
# Import Cleaned Data
df = pd.read_csv('yelp_cleaned.csv')

# Drop Unnamed Columns
cols_to_drop = [c for c in df.columns if "Unnamed" in c]
df = df.drop(columns=cols_to_drop)

In [87]:
# Convert Price ($$$) to Numeric
map_price = {'$': 1, '$$': 2, '$$$': 3, '$$$$': 4}
df['price_level'] = df['price'].map(map_price)

In [88]:
# Convert String Columns to Lists
list_cols = ['categories', 'categories_alias', 'transactions']

for col in list_cols:
    df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [89]:
# Get Primary Cuisine by Selecting First Category
df['primary_cuisine'] = df['categories'].apply(
    lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None
)

In [90]:
# Standardizing Cuisine Labels with the Most Inconsistencies
cuisine_map = {
    'New American': 'American (New)',
    'American (New)': 'American (New)',
    'newamerican': 'American (New)',

    'Seafood': 'Seafood',
    'seafood': 'Seafood',

    'Bars': 'Bar',
    'bars': 'Bar',

    'Breakfast & Brunch': 'Breakfast/Brunch',
}

df['primary_cuisine_clean'] = df['primary_cuisine'].replace(cuisine_map)

In [91]:
# Finding if Restaurants are Chains
chain_counts = df.groupby('name')['city'].nunique()
chains = chain_counts[chain_counts > 1].index.tolist()

df['is_chain'] = df['name'].isin(chains)

In [92]:
# Mapping all the Primary Cuisines into 12 Major Cuisines
def map_cuisine(c):
    if c is None:
        return "Other"
    c_lower = c.lower()

    # American
    if any(k in c_lower for k in [
        "american", "new american", "trad american", "bbq", "barbeque",
        "burgers", "steakhouses", "chicken", "wings", "hot dogs",
        "southern", "diner", "comfort", "soul food", "hawaiian", "cajun/creole",
        "smokehouse"
    ]):
        return "American"

    # Asian
    if any(k in c_lower for k in [
        "asian", "chinese", "szechuan", "taiwanese", "thai", "korean",
        "japanese", "sushi", "ramen", "viet", "vietnamese", "malaysian",
        "mongolian", "filipino", "himalayan", "nepalese", "indian",
        "pakistani", "poke", "hot pot", "dim sum", "cantonese", "burmese",
        "laotian", "izakaya", "cambodian", "shanghainese", "indonesian",
        "teppanyaki"
    ]):
        return "Asian"

    # European
    if any(k in c_lower for k in [
        "italian", "french", "creperie", "polish", "european",
        "mediterranean", "greek", "spanish", "tapas", "irish",
        "german", "ukrainian", "portuguese", "fish & chips", "british",
        "pasta shops", "russian", "belgian", "scandinavian"
    ]):
        return "European"

    # Latin American
    if any(k in c_lower for k in [
        "mexican", "latin", "latin american", "brazilian", "peruvian",
        "colombian", "cuban", "caribbean", "empanadas",
        "venezuelan", "dominican", "salvadoran", "argentine", "tacos", "tex-mex",
        "puerto rican", "haitian", "honduran"
    ]):
        return "Latin American"

    # Middle Eastern
    if any(k in c_lower for k in [
        "middle eastern", "afghan", "lebanese", "halal", "turkish", "falafel", "persian",
        "arabic", "egyptian", "sicilian", "syrian"
    ]):
        return "Middle Eastern"

    # African
    if any(k in c_lower for k in [
        "african", "ethiopian", "moroccan", "nigerian"
    ]):
        return "African"

    # Dessert/Bakery
    if any(k in c_lower for k in [
        "dessert", "bakery", "ice cream", "frozen yogurt", "donuts",
        "pastry", "cake", "waffles", "bagels", "bakeries", "acai bowls",
        "shaved ice", "gelato", "chocolatiers & shops"
    ]):
        return "Dessert/Bakery"

    # Cafe/Coffee
    if any(k in c_lower for k in [
        "coffee", "cafe", "tea", "juice bar"
    ]):
        return "Café/Coffee"

    # Bar/Pub
    if any(k in c_lower for k in [
        "bar", "brew", "pub", "lounges", "wine", "beer garden", "taproom"
    ]):
        return "Bar/Pub"

    # Seafood
    if "seafood" in c_lower:
        return "Seafood"

    # Vegetarian/Vegan
    if any(k in c_lower for k in ["vegetarian", "vegan", "gluten-free"]):
        return "Vegetarian/Vegan"

    # Fast Food
    if any(k in c_lower for k in [
        "fast food", "sandwiches", "pizza", "buffet", "food truck",
        "food stand", "food court", "salad", "poke"
    ]):
        return "Fast Food"

    return "Other"

df['cuisine_group'] = df['primary_cuisine'].apply(map_cuisine)

In [93]:
# Has the most count in the other category
ambiguous_primary = [
    "Breakfast & Brunch",
    "Delis",
    "Restaurants",
    "Noodles",
    "Soup"
    "Wraps",
    "Specialty Food",
    "Pop-Up Restaurants",
    "Street Vendors"
]

In [94]:
# Find a Secondary Cuisine for Ambiguous Primary Cuisines
def find_secondary_cuisine(row):
    primary = row['primary_cuisine']
    cats = row['categories']

    # Keep primary if meaningful
    if primary not in ambiguous_primary:
        return primary

    # Search other categories in the list
    for c in cats[1:]:  # skip the first
        if c is None:
            continue
        if map_cuisine(c) != "Other":
            return c  # use this as improved primary

    # If nothing better, go back to primary
    return primary

df['primary_cuisine_improved'] = df.apply(find_secondary_cuisine, axis=1)
df['cuisine_group'] = df['primary_cuisine_improved'].apply(map_cuisine)

In [95]:
# Glimpse of df
df.head(5)

,name,rating,price,categories,city,state,review_count,transactions,name_alias,categories_alias,price_level,primary_cuisine,primary_cuisine_clean,is_chain,cuisine_group,primary_cuisine_improved
0,The Poppy and Parliament,4.4,$$,"[Pubs, Breakfast & Brunch, British]",Huntsville,AL,322,[delivery],the-poppy-and-parliament-huntsville,"[pubs, breakfast_brunch, british]",2.0,Pubs,Pubs,True,Bar/Pub,Pubs
1,Char Restaurant,4.3,$$,"[Steakhouses, Seafood, Bars]",Huntsville,AL,350,"[delivery, pickup]",char-restaurant-huntsville-3,"[steak, seafood, bars]",2.0,Steakhouses,Steakhouses,True,American,Steakhouses
2,Connors Steak & Seafood,4.3,$$$,"[Steakhouses, Salad, Seafood]",Huntsville,AL,823,"[delivery, pickup]",connors-steak-and-seafood-huntsville,"[steak, salad, seafood]",3.0,Steakhouses,Steakhouses,True,American,Steakhouses
3,Melt - Huntsville,4.3,$$,"[Sandwiches, Salad, Comfort Food]",Huntsville,AL,345,"[delivery, pickup]",melt-huntsville-huntsville-7,"[sandwiches, salad, comfortfood]",2.0,Sandwiches,Sandwiches,True,Fast Food,Sandwiches
4,Cotton Row Restaurant,4.0,$$$,[New American],Huntsville,AL,399,[delivery],cotton-row-restaurant-huntsville,[newamerican],3.0,New American,American (New),True,American,New American


In [96]:
# Gives Summary Stats for Each City
city_summary = df.groupby(['state', 'city']).agg(
    avg_rating = ('rating', 'mean'),
    rating_variance = ('rating', 'var'),
    avg_price = ('price_level', 'mean'),
    n_restaurants = ('name', 'count'),
    avg_review_count = ('review_count', 'mean')
).reset_index()

city_summary.head()

,state,city,avg_rating,rating_variance,avg_price,n_restaurants,avg_review_count
0,AK,Anchorage,4.186500,0.160370,2.006849,200,245.125000
1,AK,Bethel,2.284211,3.026959,2.000000,19,4.000000
2,AK,Fairbanks,3.607500,1.447833,1.848739,200,75.025000
3,AK,Juneau,3.128696,2.523994,1.859375,115,74.913043
4,AK,Kenai,2.851961,3.537966,1.608696,102,29.882353


In [97]:
df.to_csv("yelp_processed.csv", index=False)
city_summary.to_csv("city_summary.csv", index=False)